# **Overview**
Indian Liver Patient Dataset (ILPD) from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/). Perform feature engineering, data preprocessing, dimensionality reduction, and logistic regression to predict whether a patient has liver disease.

In [ ]:

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
pd.set_option('display.max_columns', None)

## **Description of the Dataset**


* The ILPD comprises 583 patient records with 10 biochemical/demographic features and a target feature that are listed below:

The dataset has the following columns:

- `Age` (Integer): Patient's age.
- `Gender` (Categorical: Male/Female).
- `TB` (Total Bilirubin, Continuous).
- `DB` (Direct Bilirubin, Continuous).
- `Alkphos` (Alkaline Phosphotase, Integer).
- `Sgpt` (Alamine Aminotransferase, Integer).
- `Sgot` (Aspartate Aminotransferase, Integer).
- `TP` (Total Proteins, Continuous).
- `ALB` (Albumin, Continuous).
- `A/G Ratio` (Albumin and Globulin Ratio, Continuous).
- `Selector` (Binary: 1 = Liver patient, 2 = Non-liver patient)

### **Import libraries, Load the Dataset, and Create a Header**


In [ ]:

column_names = ['Age', 'Gender', 'TB', 'DB', 'Alkphos', 'Sgpt', 'Sgot', 'TP', 'ALB', 'A/G Ratio', 'Selector']
df = pd.read_csv("ILPD.csv", names = column_names)
df['Target'] = df['Selector'].apply(lambda x:1 if x == 1 else 0)

df = df.drop(columns=['Selector'])

column_namesnew = ['Age', 'Gender', 'TB', 'DB', 'Alkphos', 'Sgpt', 'Sgot', 'TP', 'ALB', 'A/G Ratio', 'Target']
df.columns = column_namesnew
df.head(5)

### **Identify columns with missing data (if any) and then determine an appropriate strategy for each**



In [ ]:
before_NA = df.isnull().sum()

numerical = ['Age', 'TB', 'DB', 'Alkphos', 'Sgpt', 'Sgot', 'TP', 'ALB', 'A/G Ratio']
df[numerical] = df[numerical].fillna(df[numerical].median())


df['Gender'].fillna(df['Gender'].mode())
df['Gender'].fillna(df['Gender'].mode())

after_NA = df.isnull().sum()

In [ ]:



assert after_NA.sum() == 0, "Check for missing values."


print("No missing values.")





### **One-hot Encoding for Categorical Variables**



In [ ]:


df['Gender'] = df['Gender'].replace({'Male': 1, 'Female': 0})
df


### **Feature Extraction**



In [ ]:

df['Bilirubin Ratio'] = df['DB'] / df['TB']
df['ALT/AST Ratio'] = df['Sgpt'] / df['Sgot']


### **Standardization**



In [ ]:


scaler = StandardScaler()
df[numerical] = scaler.fit_transform(df[numerical])
df.head()


### **Engineering Ordinal Features**

In [ ]:

age_min = df['Age'].min()
age_max = df['Age'].max()


age_bins = sorted(set([age_min, 30, 50, age_max]))
age_labels = ['Young', 'Middle-Aged', 'Senior']


if len(age_bins) < 3:
    age_bins = [age_min, (age_min + age_max) / 2, age_max]
    age_labels = ['Young', 'Senior']


df['age_group'] = pd.cut(df['Age'], bins=age_bins, labels = age_labels, include_lowest = True)
df['age_ord'] = df['age_group'].apply(lambda x:1 if x == 'Young' else (2 if x == 'Middle-Aged' else 3))
df['age_ord'].fillna(df['age_ord'].mode())
df['Age'].fillna(df['Age'].median())
# Map ordinal categories
# Display first few rows
df.head()

### **Dimensionality Reduction**

In [ ]:



num_df = df.drop(columns=['Target', 'Gender', 'age_group', 'age_ord'])
X = num_df
y = df['Target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred_before = model.predict(X_test)
accuracy_bef= accuracy_score(y_test, y_pred_before)
print("Accuracy before:", accuracy_bef)

scaler = StandardScaler()
scaled = scaler.fit_transform(X)

pca = PCA(n_components=4)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

model.fit(X_train_pca, y_train)
y_pred_after_4 = model.predict(X_test_pca)
accuracy_after_4 = accuracy_score(y_test, y_pred_after_4)
print("Accuracy after:", accuracy_after_4)

pca = PCA(n_components=2)
X_train_pca_2 = pca.fit_transform(X_train)
X_test_pca_2 = pca.transform(X_test)

model.fit(X_train_pca_2, y_train)
y_pred_after_2 = model.predict(X_test_pca_2)
accuracy_after_2 = accuracy_score(y_test, y_pred_after_2)
print("Accuracy after:",accuracy_after_2)



## **Logistic Regression Model**

* Train a logistic regression model using the processed dataset.


In [ ]:

df = pd.get_dummies(df, columns=['age_group'])

X = df.drop(columns=['Target'])
y1 = df['Target']

X_train, X_test, y_train, y = train_test_split(X, y1, test_size=0.2, random_state=42)
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

heatmap = confusion_matrix(y, y_pred)
plt.figure(figsize=(6,4))
sns.heatmap(heatmap)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix Heatmap")
plt.show()
# Evaluate model
print("Accuracy:", accuracy_score(y, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y, y_pred))
print("Classification Report:\n", classification_report(y, y_pred))


#coeffients
feature = X.columns
coefficient = model.coef_[0]
coefdf = pd.DataFrame({'Feature': feature, 'Coefficient': coefficient})
coefdf['Abs_Coefficient'] =  coefdf['Coefficient'].abs()
coefdf = coefdf.sort_values(by='Abs_Coefficient')


plt.figure(figsize=(10,6))
sns.barplot(x='Abs_Coefficient', y='Feature', data = coefdf)
plt.title("Features Based on Logistic Regression Coefficients")
plt.xlabel("Absolute Coefficient Value")
plt.ylabel("Feature")
plt.show()


## **Visualisation**
* Plot the distribution of Total Bilirubin (TB) for liver and non-liver patients.



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

liver_patients = df[df['Target'] == 1]
non_liver_patients = df[df['Target'] == 0]
plt.figure(figsize=(10, 6))
plt.hist(liver_patients['TB'], color='red', label='Liver Patients', alpha = 0.5)
plt.hist(non_liver_patients['TB'], color='blue', label='Non-Liver Patients')
plt.xlabel('Total Bilirubin (TB)')
plt.ylabel('Frequency')
plt.title('Total Bilirubin (TB) for liver and non-liver patient')
plt.legend()
plt.show()

**Based on the confusion matrix and classification report, this model perform pretty well to identifying liver patients based on the TB. As the F1 score is high as 0.85, this model is precise for Liver or Class 1. The recall for non-liver patients is at 0.30 indicating that it fails to correctly diagnore non-liver patients (about 21 false positives). The model is incorrectly classifies non-liver patients as
liver patients and increases the number of false positives that may be a problem in diagnosis. But, false negatives are more concerning as failing to diagnose a patient has more severe consequences than wrongly diagnosing that a patient has liver disease.**




**TP, DB, Sgot, Sgpt, and ALB are the features that seem to have the strongest impact on the predictions pf liver disearse based on the logistic regression coefficients. These findings make sense because the five features are the main trackers to ensure one's liver health is in good condition. TP, DB, Sgot, Sgpt, and ALB help assess liver health. FOr example, SGOT and SGPT are two common enzymes produced by the liver. TP indicates the total protein.**

**These transformations to create engineered features can help the model by observing important relationship to increase accuracy of diagonsis for liver disease. But, this may result in bias in the predictions due to improper binning and not being properly scaled. This may cause the new variables to not have accurate predictions based on the data.**

**Both distributions are right tailed
with higher frequency data points for liver patients. This graph may indicate that those with higher TB are more likely to be liver patients. There is a noticeable difference between liver and non-liver patients (red and blue). Higher values of TB are associated with liver disease patients for most of the data distribution. Therefore, TB is a strong predictor for detecting liver disease.**